In this notebook, we will review and implement how to handle and prepare unlabeled data for injection attacks

#Install Tools and dependencies

In [ ]:
!pip install transformers torch sklearn pandas

#Load the necessary libraries

In [ ]:
#Load Libraries
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import normalize
import numpy as np

#Data Loading  

In [ ]:
# Load datasets
labeled_data = pd.read_csv('/content/Complete_Label_train_Dataset - Copy of Train_Label_Dataset1 - Train_Label_Dataset.csv')
non_labeled_data = pd.read_csv('/content/Non_Label_Dataset - Copy of Train_Label_Dataset1 - Train_Label_Dataset - Non_Label_Dataset - Copy of Train_Label_Dataset1 - Train_Label_Dataset.csv')

# Use only partial labeled data for training
partial_labeled_data = labeled_data.sample(n=150, random_state=42)


#Preprocessing

In [ ]:
# Load XLM-RoBERTa tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")
model = AutoModel.from_pretrained("xlm-roberta-base")

# Function to generate embeddings
def get_embeddings(texts):
    tokens = tokenizer(texts, padding=True, truncation=True, return_tensors="pt", max_length=512)
    with torch.no_grad():
        outputs = model(**tokens)
    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings.numpy()

#Feature Extraction

In [ ]:
# Generate embeddings for labeled and non-labeled data
print("Generating embeddings for partial labeled data...")
partial_labeled_data['embeddings'] = list(get_embeddings(partial_labeled_data['text'].tolist()))

print("Generating embeddings for non-labeled data...")
non_labeled_data['embeddings'] = list(get_embeddings(non_labeled_data['text'].tolist()))

# Normalize embeddings for cosine similarity
labeled_embeddings = normalize(np.vstack(partial_labeled_data['embeddings']))
non_labeled_embeddings = normalize(np.vstack(non_labeled_data['embeddings']))

#Cosine similarity and label assignment

In [ ]:
# Calculate similarity and assign weak labels
def assign_labels(non_labeled_embeddings, labeled_embeddings, labeled_labels):
    weak_labels = []
    for non_label_emb in non_labeled_embeddings:
        # Compute similarity with all labeled samples
        similarities = cosine_similarity([non_label_emb], labeled_embeddings)[0]
        # Assign label of the most similar labeled sample
        most_similar_idx = similarities.argmax()
        weak_labels.append(labeled_labels[most_similar_idx])
    return weak_labels

print("Assigning weak labels based on similarity...")
non_labeled_data['weak_label'] = assign_labels(
    non_labeled_embeddings,
    labeled_embeddings,
    partial_labeled_data['label'].tolist()
)

# Save the weakly labeled data
non_labeled_data.to_csv('/content/Weakly_Labeled_Non_Label_Dataset.csv', index=False)
print("Weak labels assigned and saved!")


Generating embeddings for partial labeled data...
Generating embeddings for non-labeled data...
Assigning weak labels based on similarity...
Weak labels assigned and saved!
